In [ ]:
# !pip install google-generativeai pandas
# 주피터 노트북 셀에서 '!'를 앞에 붙이면 터미널 명령어를 실행할 수 있습니다.

import google.generativeai as genai
import pandas as pd
from IPython.display import display, Markdown

import data_collector as dc # data_collector.py에서 정의한 함수를 사용합니다.

# 여기에 본인의 Google AI Studio API 키를 입력하세요.
# https://aistudio.google.com/app/apikey
API_KEY = '' # ◀◀◀◀◀ 중요: 본인의 API 키로 교체하세요!

genai.configure(api_key=API_KEY)

print("라이브러리 및 API 키 설정이 완료되었습니다.")

라이브러리 및 API 키 설정이 완료되었습니다.


In [24]:
# SK하이닉스 뉴스 데이터 수집 및 처리

from data_collector import fetch_naver_news

company_name = "SK하이닉스"

# 1. 네이버 뉴스 API에서 뉴스 데이터 가져오기
news_articles = fetch_naver_news(company_name)

# 2. 회사 이름이 포함된 뉴스만 필터링
filtered_headlines = []
for article in news_articles:
    if company_name in article['title'] or company_name in article['description']:
        filtered_headlines.append(article['title'])

# 3. 컨텍스트 데이터 구성
context_data = f"""### {company_name} 최신 뉴스
- {"\n- ".join(filtered_headlines)}
"""

print("✅ LLM에게 전달될 컨텍스트 데이터가 생성되었습니다.")
print("----------------------------------------------------")
print(context_data)

🔍 API 응답 정보: 총 796994건 중 5건 반환
✅ 네이버 뉴스 API 호출 성공: 'SK하이닉스' 관련 뉴스 5건 수집
✅ LLM에게 전달될 컨텍스트 데이터가 생성되었습니다.
----------------------------------------------------
### SK하이닉스 최신 뉴스
- [관세 타결]상호관세 낮추고 '최혜국 대우' 약속…반도체, 한숨 돌렸다
- 머스크 사로잡은 삼성전자…하반기엔 반도체 띄운다
- 미중 사이에 낀 한국…위기를 이겨낼 패권 전략은?[북적book적]
- 삼성전자 제친 SK하이닉스…대학생 선호도 1위 기업 올라
- 반도체 부진에 흔들린 삼성, 1년 새 영업익 55% 날라갔다



In [25]:
def create_report_prompt(company_name, context_data):
    """최종 프롬프트를 생성하는 함수"""
    prompt_template = f"""
# 지시문: 당신은 대한민국 최고의 증권사 애널리스트입니다. 아래 [컨텍스트 데이터]만을 사용하여, '{company_name}'에 대한 투자 리포트를 다음 [리포트 형식]에 맞춰 작성해 주세요. 컨텍스트에 없는 내용은 절대 지어내지 마세요.

# 리포트 형식
1.  **핵심 요약 (Key Highlights):** 최신 뉴스를 2~3개의 불렛 포인트로 요약.
2.  **긍정적/부정적 신호 분석:** 뉴스 내용에서 긍정적인 신호와 부정적인 신호를 각각 분석.
3.  **종합 의견:** 위 내용을 종합하여 투자 관점에서의 간단한 의견을 제시.

# 컨텍스트 데이터
{context_data}
"""
    return prompt_template

final_prompt = create_report_prompt(company_name, context_data)

print("✅ 최종 프롬프트가 성공적으로 생성되었습니다.")

✅ 최종 프롬프트가 성공적으로 생성되었습니다.


In [26]:
print("⏳ Gemini 모델을 호출하여 리포트를 생성합니다...")

# 개발/테스트 단계에서는 빠른 Flash 모델을 사용하는 것을 권장합니다.
model = genai.GenerativeModel('gemini-2.5-flash') 

response = model.generate_content(final_prompt)

print("✅ 리포트 생성이 완료되었습니다!")
print("====================================================")

# 마크다운 형식으로 출력하여 가독성을 높입니다.
display(Markdown(response.text))

⏳ Gemini 모델을 호출하여 리포트를 생성합니다...
✅ 리포트 생성이 완료되었습니다!
✅ 리포트 생성이 완료되었습니다!


⏳ Gemini 모델을 호출하여 리포트를 생성합니다...
✅ 리포트 생성이 완료되었습니다!
✅ 리포트 생성이 완료되었습니다!


## SK하이닉스 투자 리포트

**1. 핵심 요약 (Key Highlights):**

*   반도체 산업 전반에 대한 상호관세 타결 및 '최혜국 대우' 약속으로 불확실성이 일부 해소되었습니다.
*   SK하이닉스는 삼성전자를 제치고 대학생 선호도 1위 기업에 올랐습니다.
*   반도체 업황 전반의 부진이 지속되고 있으며, 이는 주요 기업의 영업이익 감소로 나타나고 있습니다.

**2. 긍정적/부정적 신호 분석:**

*   **긍정적 신호:**
    *   **관세 타결:** 상호관세가 낮아지고 '최혜국 대우'가 약속됨에 따라 반도체 산업 전반에 걸쳐 교역 환경이 개선될 여지가 생겼습니다. 이는 SK하이닉스의 수출 및 사업 활동에 긍정적인 영향을 미칠 수 있습니다.
    *   **기업 선호도 1위:** 대학생 선호도에서 삼성전자를 제치고 1위를 차지한 것은 SK하이닉스의 기업 이미지 및 인재 유치 경쟁력 강화에 긍정적인 신호입니다. 장기적인 기업 성장에 필요한 우수 인력 확보에 유리할 수 있습니다.
*   **부정적 신호:**
    *   **반도체 업황 부진 지속:** 주요 반도체 기업의 영업이익이 크게 감소하는 등 전반적인 반도체 업황의 부진이 지속되고 있습니다. 이는 SK하이닉스 또한 어려운 시장 환경에 직면해 있음을 시사합니다.
    *   **지정학적 리스크:** 미중 갈등 사이에 낀 한국 반도체 산업의 지정학적 리스크는 여전히 존재하며, 이는 사업 환경의 불확실성을 높이는 요인으로 작용할 수 있습니다.

**3. 종합 의견:**

SK하이닉스는 반도체 업황의 전반적인 부진과 지정학적 리스크라는 도전 과제에 직면해 있습니다. 그러나 최근의 관세 타결 소식은 산업 전반의 불확실성을 일부 해소하며 긍정적인 영향을 줄 것으로 보이며, 우수한 인재 선호도 1위 기업 선정은 장기적인 기업 경쟁력 강화에 기여할 수 있습니다. 다만, 거시적인 반도체 시장의 근본적인 회복 여부는 지속적인 관찰이 필요합니다.

In [ ]:
# 🎯 최종 투자 리포트 자동 생성 시스템

import importlib
importlib.reload(data_collector)  # 최신 코드 반영

from data_collector import create_final_investment_report

company_name = "SK하이닉스"

print("🎯 최종 투자 리포트 자동 생성 시스템")
print("="*60)
print("📊 분석 구성 요소:")
print("  📰 네이버 뉴스 API - 최신 뉴스 5건")
print("  🏆 DART 공시 API - 중요도별 분류")
print("  🤖 Gemini AI - 종합 분석 리포트")
print("="*60)

# 최종 리포트 데이터 생성
try:
    print(f"\n🚀 {company_name} 투자 리포트 자동 생성 시작...")
    
    # 스마트 리포트 생성 (뉴스 + 중요도별 공시)
    report_data = create_final_investment_report(company_name)
    
    if report_data:
        print(f"\n✅ 리포트 데이터 생성 완료!")
        print(f"  📅 생성 시간: {report_data['generation_time']}")
        print(f"  🏢 분석 대상: {report_data['company_name']}")
        
        # 컨텍스트 데이터 미리보기
        context_preview = report_data['context_data'][:500]
        print(f"\n📋 컨텍스트 데이터 미리보기:")
        print("─" * 50)
        print(f"{context_preview}...")
        print("─" * 50)
        
        # 이제 이 데이터를 Gemini에게 전달
        final_prompt = report_data['final_prompt']
        context_data = report_data['context_data']
        
        print(f"\n🎯 최종 프롬프트 준비 완료!")
        print(f"  📝 프롬프트 길이: {len(final_prompt):,} 문자")
        print(f"  📊 컨텍스트 길이: {len(context_data):,} 문자")
        
    else:
        print("❌ 리포트 데이터 생성 실패")
        
except Exception as e:
    print(f"❌ 오류 발생: {e}")
    final_prompt = None
    context_data = None

print(f"\n" + "="*60)
print("💡 다음 단계: 아래 셀에서 Gemini AI로 최종 분석 실행")

In [ ]:
# 🤖 Gemini AI 최종 투자 리포트 생성

if 'final_prompt' in globals() and final_prompt:
    print("🤖 Gemini AI로 최종 투자 리포트 생성 중...")
    print("⏳ 분석 진행 중... (약 10-30초 소요)")
    
    try:
        # Gemini 모델 호출
        model = genai.GenerativeModel('gemini-2.5-flash')
        response = model.generate_content(final_prompt)
        
        print("✅ 🎉 최종 투자 리포트 생성 완료!")
        print("="*60)
        
        # 마크다운 형식으로 아름답게 출력
        display(Markdown(f"""
# 📊 {company_name} 투자 분석 리포트

{response.text}

---
**📅 리포트 생성 정보**
- 분석 대상: {company_name}
- 생성 시간: {report_data['generation_time']}
- 데이터 소스: 네이버 뉴스 API + DART 공시 API
- AI 모델: Google Gemini 2.5 Flash
- 분석 기준: 최신 뉴스 + 중요도별 공시 (90일)

**⚠️ 투자 유의사항**
- 본 리포트는 공개된 뉴스와 공시 정보만을 바탕으로 작성되었습니다.
- 투자 결정은 반드시 본인의 판단과 추가적인 정보 수집을 통해 신중히 하시기 바랍니다.
- 과거의 성과가 미래를 보장하지 않습니다.
        """))
        
    except Exception as e:
        print(f"❌ AI 분석 실패: {e}")
        print("💡 API 키 설정이나 네트워크 연결을 확인해주세요.")
        
else:
    print("❌ 최종 프롬프트가 준비되지 않았습니다.")
    print("💡 위 셀을 먼저 실행해주세요.")

# 🎯 최종 투자 리포트 자동 생성 시스템 완성! 

## 🚀 주요 기능

### 1. **스마트 데이터 수집**
- **네이버 뉴스 API**: 최신 뉴스 5건 실시간 수집
- **DART 공시 API**: 90일간 중요 공시만 선별 분석
- **CSV 캐싱**: 11만+ 기업 정보 빠른 검색

### 2. **중요도별 공시 분류** 🏆📊
- **🏆 1순위**: 실적보고서, 영업실적 → 기업 성과 핵심
- **🚀 2순위**: M&A, 증자, 자사주 → 주가 영향 중대 결정  
- **📈 3순위**: 대형계약, 설비투자 → 미래 성장 동력
- **⚠️ 리스크**: 소송, 주주변경 → 위험 신호

### 3. **AI 기반 종합 분석**
- **Google Gemini 2.5**: 최신 AI 모델 활용
- **전문 프롬프트**: 증권사 애널리스트 수준 분석
- **구조화된 리포트**: 투자 포인트 → 리스크 → 최종 의견

## 📋 주요 함수들

```python
# 1. 중요도별 공시 분류
classified = get_important_disclosures_by_priority("SK하이닉스")

# 2. 스마트 컨텍스트 생성 (뉴스 + 중요 공시)
context = create_smart_company_report("SK하이닉스")

# 3. 최종 리포트 자동 생성 (올인원)
report = create_final_investment_report("SK하이닉스")

# 4. Gemini AI 분석
model = genai.GenerativeModel('gemini-2.5-flash')
response = model.generate_content(report['final_prompt'])
```

## 🎯 사용법

1. **API 키 설정** (한 번만)
   - 네이버 개발자 센터: 뉴스 검색 API
   - DART 공시시스템: 공시 정보 API (선택사항)
   - Google AI Studio: Gemini API

2. **간단한 실행** (1줄 코드)
   ```python
   report_data = create_final_investment_report("원하는회사명")
   ```

3. **AI 분석 실행**
   ```python
   response = model.generate_content(report_data['final_prompt'])
   ```

## 🌟 완성된 시스템의 장점

- ✅ **완전 자동화**: 회사명 입력만으로 전체 프로세스 실행
- ✅ **전문성**: 증권사 수준의 체계적 분석 구조
- ✅ **실시간**: 최신 뉴스 + 공시 정보 실시간 반영
- ✅ **효율성**: 중요한 정보만 선별해서 빠른 분석
- ✅ **확장성**: 어떤 상장 기업이든 동일한 품질로 분석

이제 **전문가 수준의 투자 리포트**를 **1분 안에** 자동으로 생성할 수 있습니다! 🎉

In [ ]:
# ✅ 수정된 고유번호 처리 기능 테스트

import importlib
importlib.reload(data_collector)

from data_collector import get_corp_info_fast, fetch_dart_disclosures

print("🔧 수정된 data_collector.py 고유번호 처리 테스트")
print("="*60)

# 1. SK하이닉스 정보 조회 (수정된 함수)
company_name = "SK하이닉스"
print(f"1️⃣ {company_name} 고유번호 조회 테스트")

corp_info = get_corp_info_fast(company_name)
if corp_info:
    print(f"✅ 회사명: {corp_info['corp_name']}")
    print(f"✅ 고유번호: {corp_info['corp_code']} (길이: {len(corp_info['corp_code'])}자리)")
    print(f"✅ 타입: {type(corp_info['corp_code'])}")
    
    # 2. DART API 테스트 (DART API 키가 있는 경우에만)
    print(f"\n2️⃣ DART API 호출 테스트 (고유번호: {corp_info['corp_code']})")
    
    # API 키가 설정되어 있는지 확인
    import inspect
    source = inspect.getsource(data_collector.fetch_dart_disclosures)
    
    if 'DART_API_KEY = ""' in source:
        print("⚠️ DART API 키가 설정되지 않아 실제 API 호출은 생략합니다.")
        print("💡 DART API 키 설정 시 실제 공시 데이터를 가져올 수 있습니다.")
    else:
        print("📡 DART API 호출 시도...")
        test_disclosures = fetch_dart_disclosures(corp_info['corp_code'], page_count=3)
        
        if test_disclosures:
            print(f"🎉 성공! {len(test_disclosures)}건의 공시 정보 수집")
            for i, disc in enumerate(test_disclosures[:2], 1):
                print(f"  {i}. {disc.get('report_nm', 'N/A')}")
        else:
            print("❌ API 호출 실패 또는 데이터 없음")
    
else:
    print("❌ 회사 정보를 찾을 수 없습니다.")

print()
print("="*60)
print("🎯 결론:")
print("✅ 고유번호 처리 문제 해결 완료!")
print("✅ CSV에서 문자열로 읽어와 8자리 형식 보장")
print("✅ DART API 호출 시 올바른 형식으로 전달")
print("✅ 생산자-소비자 아키텍처 구축 준비 완료!")